In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('linreg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [7]:
train = spark.read.format('libsvm').load('../spark-ml/sample_linear_regression_data.txt')

In [8]:
train.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [11]:
lrModel = lr.fit(train)

In [13]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [14]:
lrModel.intercept

0.14228558260358093

In [18]:
summary = lrModel.summary
summary

In [19]:
summary.r2

0.027839179518600154

In [20]:
summary.rootMeanSquaredError

10.16309157133015

In [25]:
# Data should be slit in train and test sets
all_data = spark.read.format('libsvm').load('../spark-ml/sample_linear_regression_data.txt')
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [34]:
train.count()

501

In [35]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                365|
|   mean|0.18706827835132536|
| stddev| 10.135407688993295|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [36]:
# Lets test the model with the split
model = lr.fit(train_data)

In [37]:
test_results = model.evaluate(test_data)

In [38]:
test_results.rootMeanSquaredError

10.72347882837474

In [40]:
unlabeled_data = train_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [42]:
predictions = model.transform(unlabeled_data)
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -1.7987080555170931|
|(10,[0,1,2,3,4,5,...|  0.9349150964849796|
|(10,[0,1,2,3,4,5,...| -2.9075634686701837|
|(10,[0,1,2,3,4,5,...| -1.8001754528592118|
|(10,[0,1,2,3,4,5,...|-0.24868245310383016|
|(10,[0,1,2,3,4,5,...|  2.2321566704029876|
|(10,[0,1,2,3,4,5,...|  0.4806584690573762|
|(10,[0,1,2,3,4,5,...|  1.2580448740475274|
|(10,[0,1,2,3,4,5,...|   1.375656228170007|
|(10,[0,1,2,3,4,5,...| 0.24527143110987115|
|(10,[0,1,2,3,4,5,...| 0.22108810527458647|
|(10,[0,1,2,3,4,5,...| -2.8973487499940442|
|(10,[0,1,2,3,4,5,...|  0.8216738637179578|
|(10,[0,1,2,3,4,5,...|  0.5743217080795638|
|(10,[0,1,2,3,4,5,...|-0.19506416754140318|
|(10,[0,1,2,3,4,5,...| -1.8371103538285918|
|(10,[0,1,2,3,4,5,...| -1.2316581979679169|
|(10,[0,1,2,3,4,5,...|  -1.369807777893501|
|(10,[0,1,2,3,4,5,...|  -1.109141434757223|
|(10,[0,1,2,3,4,5,...|   0.74157